In [1]:
#credits to https://www.kaggle.com/remekkinas/yolox-training-pipeline-cots-dataset-lb-0-507

In [2]:
!nvidia-smi

Mon Feb  7 11:11:47 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.119.04   Driver Version: 450.119.04   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%%time
%pip install -U -q wandb
%pip install -q imagesize
%pip install -q ensemble-boxes
%pip install -q git+https://github.com/ildoonet/pytorch-gradual-warmup-lr.git

In [ ]:
%%time
#!git clone https://github.com/Megvii-BaseDetection/YOLOX -q

!mkdir /kaggle/working/YOLOX
%cp -rf ../input/yolox-wandb/* /kaggle/working/YOLOX

%cd YOLOX
%pip install -U -q pip && pip install -q -r requirements.txt
%pip install -q -v -e . 

%cd /kaggle/working/

In [6]:
%cp -r /kaggle/input/pytorch-underwater-image-enhancement/PyTorch-Underwater-Image-Enhancement /kaggle/working/

In [ ]:
%pip install -q 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

In [ ]:
%pip uninstall -q scikit-learn -y
%pip install -q --pre --extra-index https://pypi.anaconda.org/scipy-wheels-nightly/simple scikit-learn
%python -c "import sklearn; sklearn.show_versions()"

In [9]:
import os
import wandb
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
os.environ['WANDB_API_KEY'] = user_secrets.get_secret("WANDB_API_KEY")

#wandb.init(project="Great Barrier Reef - Kaggle", entity="yovin_y")

In [10]:
import sys
import ast
import json
import random
import torch
import importlib
import cv2 
import shutil
import numpy as np
import imagesize
import pandas as pd

from shutil import copyfile
from tqdm.notebook import tqdm
tqdm.pandas()
from sklearn.model_selection import GroupKFold, StratifiedGroupKFold, StratifiedKFold
from PIL import Image
from string import Template
from IPython.display import display

from joblib import Parallel, delayed
from sceneRadianceCLAHE import RecoverCLAHE
from sceneRadianceHE import RecoverHE

import warnings
warnings.filterwarnings("ignore")
sys.path.append('../input/underwater-image-enhancement/Underwater Image Enhancement/CLAHE')


In [ ]:
# check Torch and CUDA version
print(f"Torch: {torch.__version__}")
!nvcc --version

In [12]:
EPOCHS = 15
N_FOLD = 5
FOLD = 0
SEED = 42
IMG_SIZE = '(1440, 1920)'
BATCH_SIZE = '8'
YOLOX_MODEL_ARCH = 'MEDIUM'
USE_CUSTOM_CSV = True
USE_CUSTOM_DATA = True
USE_PSEUDO_DATA = False
REMOVE_NOBBOX = True
ROOT_DIR  = '/kaggle/input/tensorflow-great-barrier-reef'
CUSTOM_CSV_DIR = '../input/cotts-processed-csv'
CUSTOM_DATA_DIR = '../input/cotts-processed-csv/train_images'
TRAIN_IMAGE_DIR = '/kaggle/dataset/images/train2017' # directory to save images
VAL_IMAGE_DIR = '/kaggle/dataset/images/val2017' # directory to save images
LABEL_DIR = '/kaggle/dataset/images/annotations' # directory to save labels

!mkdir -p {TRAIN_IMAGE_DIR}
!mkdir -p {VAL_IMAGE_DIR}
!mkdir -p {LABEL_DIR}

In [13]:
def seed_torch(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_torch(seed=SEED)

In [14]:
def get_bbox(annotations):
    bboxes = [list(annot.values()) for annot in annotations]
    return bboxes

def get_imgsize(row):
    row['width'], row['height'] = imagesize.get(row['image_path'])
    return row

def get_path(row):
    if USE_CUSTOM_CSV and not USE_CUSTOM_DATA:
        row['old_image_path'] = row['image_path']
        
    elif USE_CUSTOM_CSV and USE_CUSTOM_DATA:
        row['old_image_path'] = row['image_path']
        
        img_path = row.image_path.split('/')
        img_name = img_path[-1]
        folder_name = img_path[-2]
        
        row['image_path'] = f'{CUSTOM_DATA_DIR}/{folder_name}/{img_name}'
    else:
        row['old_image_path'] = f'{ROOT_DIR}/train_images/video_{row.video_id}/{row.video_frame}.jpg'
    return row

In [ ]:
if USE_CUSTOM_CSV and USE_PSEUDO_DATA:
    df_train = pd.read_csv(f'{CUSTOM_CSV_DIR}/processed.csv')
elif USE_CUSTOM_CSV:
    df_train = pd.read_csv(f'{CUSTOM_CSV_DIR}/train-{N_FOLD}-folds.csv')
else:
    df_train = pd.read_csv(f'{ROOT_DIR}/train.csv')
                     
df_train.head(5)

In [ ]:
#get image paths
df_train = df_train.progress_apply(get_path, axis=1)

# get annotations
df_train['annotations'] = df_train['annotations'].progress_apply(lambda x: ast.literal_eval(x))
df_train['bboxes'] = df_train.annotations.progress_apply(get_bbox)

df_train['num_bbox'] = df_train['annotations'].progress_apply(lambda x: len(x))
data = (df_train.num_bbox>0).value_counts()
print(f"No BBox: {data[0]:0.2f} | With BBox: {data[1]:0.2f}")

#exclude images without annotations
if REMOVE_NOBBOX:    
    N = 6000
    df_train = pd.concat([
        df_train[df_train['num_bbox'] > 0],
        df_train[df_train['num_bbox'] == 0].sample(N - (df_train.num_bbox>0).value_counts()[1])
    ]).sample(frac=1).reset_index(drop = True)
    
if USE_PSEUDO_DATA:
    df_train = df_train.iloc[0:8000]
    
#Image resolutions
df_train = df_train.progress_apply(get_imgsize,axis=1)
display(print("Image width: ",df_train.width.unique()), print("Image height: ", df_train.height.unique()))
display(df_train.head(5))

In [17]:
if USE_CUSTOM_CSV and not USE_PSEUDO_DATA:
    print("Using custom CSV file...")
    sgkf = StratifiedGroupKFold(n_splits = N_FOLD) 
    df_train = df_train.reset_index(drop=True)
    df_train['fold'] = -1
            
    for fold, (train_idx, val_idx) in enumerate(sgkf.split(df_train, y = df_train.has_annotations, groups=df_train.subsequence_id)):
        df_train.loc[val_idx, 'fold'] = fold

    df_train.head(5)
    
if USE_CUSTOM_CSV and USE_PSEUDO_DATA:
    print("Using custom processed CSV file...")
    sgkf = StratifiedGroupKFold(n_splits = N_FOLD) 
    df_train = df_train.reset_index(drop=True)
    df_train['fold'] = -1
            
    for fold, (train_idx, val_idx) in enumerate(sgkf.split(df_train, y = df_train.n_annotations, groups=df_train.subsequence_id)):
        df_train.loc[val_idx, 'fold'] = fold

    df_train.head(5)
    
if not USE_CUSTOM_CSV:
    gkf = GroupKFold(n_splits = N_FOLD) 
    df_train = df_train.reset_index(drop=True)
    df_train['fold'] = -1
    for fold, (train_idx, val_idx) in enumerate(gkf.split(df_train, y = df_train.video_id.tolist(), groups=df_train.sequence)):
        df_train.loc[val_idx, 'fold'] = fold

    df_train.head(5)
    
display(print(df_train.fold.value_counts()))

Using custom CSV file...
0    1201
3    1200
1    1200
4    1200
2    1199
Name: fold, dtype: int64


None

In [18]:
def make_copy(row, mode):
    if mode == "train":
        copyfile(f'{row.image_path}', f'{TRAIN_IMAGE_DIR}/{row.image_id}.jpg')
    elif mode == "val":
        copyfile(f'{row.image_path}', f'{VAL_IMAGE_DIR}/{row.image_id}.jpg')
    return

In [ ]:
train_images = df_train[df_train['fold'] != FOLD]
val_images = df_train[df_train['fold'] == FOLD]

_ = Parallel(n_jobs=-1, backend='threading')(delayed(make_copy)(row, mode="train") for index, row in tqdm(train_images.iterrows(), total=train_images.shape[0]))
_ = Parallel(n_jobs=-1, backend='threading')(delayed(make_copy)(row, mode="val") for index, row in tqdm(val_images.iterrows(), total=val_images.shape[0]))

In [21]:
print(f'Number of training files: {len(os.listdir(TRAIN_IMAGE_DIR))}')
print(f'Number of validation files: {len(os.listdir(VAL_IMAGE_DIR))}')

Number of training files: 4799
Number of validation files: 1201


In [22]:
def save_annot_json(json_annotation, filename):
    with open(filename, 'w') as f:
        output_json = json.dumps(json_annotation)
        f.write(output_json)

In [23]:
annotion_id = 0

In [24]:
def dataset2coco(df, dest_path):
    
    global annotion_id
    
    annotations_json = {
        "info": [],
        "licenses": [],
        "categories": [],
        "images": [],
        "annotations": []
    }
    
    info = {
        "year": "2021",
        "version": "1",
        "description": "COTS dataset - COCO format",
        "contributor": "",
        "url": "https://kaggle.com",
        "date_created": "2021-11-30T15:01:26+00:00"
    }
    annotations_json["info"].append(info)
    
    lic = {
            "id": 1,
            "url": "",
            "name": "Unknown"
        }
    annotations_json["licenses"].append(lic)

    classes = {"id": 0, "name": "starfish", "supercategory": "none"}

    annotations_json["categories"].append(classes)

    
    for ann_row in df.itertuples():
            
        images = {
            "id": ann_row[0],
            "license": 1,
            "file_name": ann_row.image_id + '.jpg',
            "height": ann_row.height,
            "width": ann_row.width,
            "date_captured": "2021-11-30T15:01:26+00:00"
        }
        
        annotations_json["images"].append(images)
        
        bbox_list = ann_row.bboxes
        
        for bbox in bbox_list:
            b_width = bbox[2]
            b_height = bbox[3]
            
            if (bbox[0] + bbox[2] > 1280):
                b_width = bbox[0] - 1280 
            if (bbox[1] + bbox[3] > 720):
                b_height = bbox[1] - 720 
                
            image_annotations = {
                "id": annotion_id,
                "image_id": ann_row[0],
                "category_id": 0,
                "bbox": [bbox[0], bbox[1], b_width, b_height],
                "area": bbox[2] * bbox[3],
                "segmentation": [],
                "iscrowd": 0
            }
            
            annotion_id += 1
            annotations_json["annotations"].append(image_annotations)
        
        
    print(f"Dataset COTS annotation to COCO json format completed! Files: {len(df)}")
    return annotations_json

In [25]:
# Convert COTS dataset to JSON COCO
train_annot_json = dataset2coco(df_train[df_train['fold'] != FOLD], TRAIN_IMAGE_DIR)
val_annot_json = dataset2coco(df_train[df_train['fold'] == FOLD], VAL_IMAGE_DIR)

# Save converted annotations
save_annot_json(train_annot_json, f"{LABEL_DIR}/train.json")
save_annot_json(val_annot_json, f"{LABEL_DIR}/valid.json")

Dataset COTS annotation to COCO json format completed! Files: 4799
Dataset COTS annotation to COCO json format completed! Files: 1201


In [26]:
%cd YOLOX

/kaggle/working/YOLOX


In [27]:
if YOLOX_MODEL_ARCH == 'SMALL':
    config_file_template = '''

#!/usr/bin/env python3
# -*- coding:utf-8 -*-
# Copyright (c) Megvii, Inc. and its affiliates.

import os

from yolox.exp import Exp as MyExp


class Exp(MyExp):
    def __init__(self):
        super(Exp, self).__init__()
        self.depth = 0.33
        self.width = 0.50
        self.exp_name = os.path.split(os.path.realpath(__file__))[1].split(".")[0]
        
        # Define yourself dataset path
        self.data_dir = "/kaggle/dataset/images"
        self.train_ann = "train.json"
        self.val_ann = "valid.json"

        self.num_classes = 1

        self.max_epoch = $max_epoch
        self.data_num_workers = 2
        self.eval_interval = 1
        
        self.mosaic_prob = 0.1 #1.0
        self.mixup_prob = 0.1 #1.0
        self.hsv_prob = 0.4 #1.0
        self.flip_prob = 0.5
        self.no_aug_epochs = 2
        
        self.input_size = $img_size
        self.mosaic_scale = (0.5, 1.5)
        #self.random_size = (10, 20)
        self.multiscale_range = 0
        self.test_size = $img_size
'''

In [28]:
if YOLOX_MODEL_ARCH == 'NANO':
    config_file_template = '''

#!/usr/bin/env python3
# -*- coding:utf-8 -*-
# Copyright (c) Megvii, Inc. and its affiliates.

import os

import torch.nn as nn

from yolox.exp import Exp as MyExp


class Exp(MyExp):
    def __init__(self):
        super(Exp, self).__init__()
        self.depth = 0.33
        self.width = 0.25
        self.input_size = (416, 416)
        self.mosaic_scale = (0.5, 1.5)
        self.random_size = (10, 20)
        self.test_size = (416, 416)
        self.exp_name = os.path.split(
            os.path.realpath(__file__))[1].split(".")[0]
        self.enable_mixup = False

        # Define yourself dataset path
        self.data_dir = "/kaggle/dataset/images/"
        self.train_ann = "train.json"
        self.val_ann = "valid.json"

        self.num_classes = 1

        self.max_epoch = $max_epoch
        self.data_num_workers = 2
        self.eval_interval = 1

    def get_model(self, sublinear=False):
        def init_yolo(M):
            for m in M.modules():
                if isinstance(m, nn.BatchNorm2d):
                    m.eps = 1e-3
                    m.momentum = 0.03

        if "model" not in self.__dict__:
            from yolox.models import YOLOX, YOLOPAFPN, YOLOXHead
            in_channels = [256, 512, 1024]
            # NANO model use depthwise = True, which is main difference.
            backbone = YOLOPAFPN(self.depth,
                                 self.width,
                                 in_channels=in_channels,
                                 depthwise=True)
            head = YOLOXHead(self.num_classes,
                             self.width,
                             in_channels=in_channels,
                             depthwise=True)
            self.model = YOLOX(backbone, head)

        self.model.apply(init_yolo)
        self.model.head.initialize_biases(1e-2)
        return self.model

'''

In [29]:
if YOLOX_MODEL_ARCH == 'MEDIUM':
    config_file_template = '''

#!/usr/bin/env python3
# -*- coding:utf-8 -*-
# Copyright (c) Megvii, Inc. and its affiliates.

import os

from yolox.exp import Exp as MyExp


class Exp(MyExp):
    def __init__(self):
        super(Exp, self).__init__()
        self.depth = 0.67
        self.width = 0.75
        self.exp_name = os.path.split(os.path.realpath(__file__))[1].split(".")[0]
        
        # Define yourself dataset path
        self.data_dir = "/kaggle/dataset/images"
        self.train_ann = "train.json"
        self.val_ann = "valid.json"

        self.num_classes = 1

        self.max_epoch = $max_epoch
        self.data_num_workers = 2
        self.eval_interval = 1
        
        self.max_norm = 5.0
        self.norm_type = 2.0
        
        # --------------- transform config ----------------- #
        
        self.mosaic_prob = 0.8 #1.0
        self.mixup_prob = 0.5 #1.0
        self.hsv_prob = 0.5 #1.0
        self.flip_prob = 0.5
        self.shear = 1.5
        self.mosaic_scale = (0.8, 2.0)
        #self.random_size = (10, 20)
        self.degrees = 10.0
        self.enable_mixup = True
        
        # --------------  training config --------------------- #
        
        self.no_aug_epochs = 2
        self.warmup_epochs = 6
        self.lr = 5e-4
        self.warmup_lr = 0.001
        self.warmup_lr_start = 5e-7
        self.min_lr_ratio = 0.05
        self.scheduler = "GradualWarmupSchedulerV2"
        self.optimizer = "adam"
        self.T_max = 12
        self.min_lr = 1e-6
        self.warmup_factor = 1
        
        self.input_size = $img_size
        self.multiscale_range = 0
        
        self.test_size = $img_size
        self.test_conf = 0.10
        self.nmsthre = 0.45
'''

In [30]:
if YOLOX_MODEL_ARCH == 'LARGE':
    config_file_template = '''

#!/usr/bin/env python3
# -*- coding:utf-8 -*-
# Copyright (c) Megvii, Inc. and its affiliates.

import os

from yolox.exp import Exp as MyExp


class Exp(MyExp):
    def __init__(self):
        super(Exp, self).__init__()
        self.depth = 1.0
        self.width = 1.0
        self.exp_name = os.path.split(os.path.realpath(__file__))[1].split(".")[0]
        
        # Define yourself dataset path
        self.data_dir = "/kaggle/dataset/images"
        self.train_ann = "train.json"
        self.val_ann = "valid.json"

        self.num_classes = 1

        self.max_epoch = $max_epoch
        self.data_num_workers = 2
        self.eval_interval = 1
        
        self.max_norm = 1.0
        self.norm_type = 2.0
        
        # --------------- transform config ----------------- #
        
        self.mosaic_prob = 0.7 #1.0
        self.mixup_prob = 0.5 #1.0
        self.hsv_prob = 0.5 #1.0
        self.flip_prob = 0.5
        self.shear = 1.5
        self.mosaic_scale = (0.8, 2.0)
        #self.random_size = (10, 20)
        self.degrees = 10.0
        self.enable_mixup = True
        
        # --------------  training config --------------------- #
        
        self.no_aug_epochs = 2
        self.warmup_epochs = 6
        self.lr = 5e-4
        self.warmup_lr = 0.001
        self.warmup_lr_start = 1e-6
        self.min_lr_ratio = 0.05
        self.scheduler = "GradualWarmupSchedulerV2"
        self.optimizer = "adam"
        self.T_max = 12
        self.min_lr = 1e-6
        self.warmup_factor = 1
        
        self.input_size = $img_size
        self.multiscale_range = 0
        
        self.test_size = $img_size
        self.test_conf = 0.10
        self.nmsthre = 0.45
'''

In [31]:
PIPELINE_CONFIG_PATH='/kaggle/working/YOLOX/cots_config.py'

pipeline = Template(config_file_template).substitute(max_epoch = EPOCHS, img_size=IMG_SIZE)

with open(PIPELINE_CONFIG_PATH, 'w') as f:
    f.write(pipeline)

In [32]:
# ./yolox/data/datasets/voc_classes.py

voc_cls = '''
VOC_CLASSES = (
  "starfish",
)
'''
with open('/kaggle/working/YOLOX/yolox/data/datasets/voc_classes.py', 'w') as f:
    f.write(voc_cls)

# ./yolox/data/datasets/coco_classes.py

coco_cls = '''
COCO_CLASSES = (
  "starfish",
)
'''
with open('/kaggle/working/YOLOX/yolox/data/datasets/coco_classes.py', 'w') as f:
    f.write(coco_cls)

# check if everything is ok    
!more /kaggle/working/YOLOX/yolox/data/datasets/coco_classes.py


COCO_CLASSES = (
  "starfish",
)


In [ ]:
#MODEL_FILE = 'yolox_s.pth'

if YOLOX_MODEL_ARCH == 'NANO':
    sh = 'wget https://github.com/Megvii-BaseDetection/storage/releases/download/0.1.1rc0yolox_nano.pth'
elif YOLOX_MODEL_ARCH == "SMALL":
    sh = 'wget https://github.com/Megvii-BaseDetection/YOLOX/releases/download/0.1.1rc0/yolox_s.pth'
elif YOLOX_MODEL_ARCH == "MEDIUM":
    sh = 'wget https://github.com/Megvii-BaseDetection/YOLOX/releases/download/0.1.1rc0/yolox_m.pth'
elif YOLOX_MODEL_ARCH == 'LARGE':
    sh = 'wget https://github.com/Megvii-BaseDetection/YOLOX/releases/download/0.1.1rc0/yolox_l.pth'

# if YOLOX_MODEL_ARCH == 'NANO':
#     sh = 'wget https://github.com/Megvii-BaseDetection/storage/releases/download/0.0.1/yolox_nano.pth'
# elif YOLOX_MODEL_ARCH == "SMALL":
#     sh = 'wget https://github.com/Megvii-BaseDetection/storage/releases/download/0.0.1/yolox_s.pth'
# elif YOLOX_MODEL_ARCH == "MEDIUM":
#     sh = 'wget https://github.com/Megvii-BaseDetection/storage/releases/download/0.0.1/yolox_m.pth'
# elif YOLOX_MODEL_ARCH == 'LARGE':
#     sh = 'wget https://github.com/Megvii-BaseDetection/storage/releases/download/0.0.1/yolox_l.pth'

MODEL_FILE = sh.split('/')[-1]

with open('script.sh', 'w') as file:
    file.write(sh)

!bash script.sh

In [34]:
!cp ./tools/train.py ./

In [ ]:
!python train.py \
    -f cots_config.py \
    -d 1 \
    -b 4 \
    --fp16 \
    -o \
    -c {MODEL_FILE} \
    --cache

In [ ]:
val_images.head(2)

In [37]:
from yolox.utils import postprocess
from yolox.data.data_augment import ValTransform

COCO_CLASSES = (
  "starfish",
)

# get YOLOX experiment
current_exp = importlib.import_module('cots_config')
exp = current_exp.Exp()

# set inference parameters
test_size = IMG_SIZE
num_classes = 1
confthre = 0.10
nmsthre = 0.45


# get YOLOX model
model = exp.get_model()
model.cuda()
model.eval()

# get custom trained checkpoint
ckpt_file = "./YOLOX_outputs/cots_config/best_ckpt.pth"
#ckpt_file = 'yolox_l.pth'
ckpt = torch.load(ckpt_file, map_location="cpu")
model.load_state_dict(ckpt["model"])

<All keys matched successfully>

In [39]:
%cd /kaggle/working

/kaggle/working


In [40]:
def yolox_inference(img, model, test_size, confthre): 
    bboxes = []
    bbclasses = []
    scores = []
    
    preproc = ValTransform(legacy = False)

    tensor_img, _ = preproc(img, None, test_size)
    tensor_img = torch.from_numpy(tensor_img).unsqueeze(0)
    tensor_img = tensor_img.float()
    tensor_img = tensor_img.cuda()

    with torch.no_grad():
        outputs = model(tensor_img)
        outputs = postprocess(
                    outputs, num_classes, confthre,
                    nmsthre, class_agnostic=True
                )

    if outputs[0] is None:
        return [], [], []
    
    outputs = outputs[0].cpu()
    bboxes = outputs[:, 0:4]

    bboxes /= min(test_size[0] / img.shape[0], test_size[1] / img.shape[1])
    bbclasses = outputs[:, 6]
    scores = outputs[:, 4] * outputs[:, 5]
    
    return bboxes, bbclasses, scores

In [42]:
from typing import List
from torchvision.ops import box_iou, box_convert

def calculate_score(
    preds: List[torch.Tensor],
    gts: List[torch.Tensor],
    iou_th: float
) -> float:
    num_tp = 0
    num_fp = 0
    num_fn = 0
    for p, gt in zip(preds, gts):
        if len(p) and len(gt):
            iou_matrix = box_iou(p, gt)
            tp = len(torch.where(iou_matrix.max(0)[0] >= iou_th)[0])
            fp = len(p) - tp
            fn = len(torch.where(iou_matrix.max(0)[0] < iou_th)[0])
            num_tp += tp
            num_fp += fp
            num_fn += fn
        elif len(p) == 0 and len(gt):
            num_fn += len(gt)
        elif len(p) and len(gt) == 0:
            num_fp += len(p)
    score = 5 * num_tp / (5 * num_tp + 4 * num_fn + num_fp)
    return score

In [43]:
def findThres(conf_th):
    gts = []
    preds = []

    for index, row in tqdm(val_images.iterrows(), total=val_images.shape[0]):
        image = cv2.imread(row.image_path)
        
        if USE_CUSTOM_DATA:
            image = RecoverCLAHE(image)
            
        annots = torch.tensor(row.bboxes)
        if len(annots)==0:
            continue

        annots = box_convert(annots, 'xywh', 'xyxy')
        
        bboxes, _, _ = yolox_inference(image, model, test_size=(1440, 1920), confthre=conf_th)

        gts.append(annots)
        preds.append(bboxes)
        
    return gts, preds

In [44]:
gts, preds = findThres(0.1)
iou_ths = np.arange(0.3, 0.85, 0.05)

scores = [calculate_score(preds, gts, iou_th) for iou_th in iou_ths]
print("Average F2 Score: ", np.mean(scores))

  0%|          | 0/1201 [00:00<?, ?it/s]

Average F2 Score:  0.553195400420743


In [45]:
print("Best_thr", iou_ths[np.argmax(scores)])
print("Best score", np.max(scores))

Best_thr 0.3
Best score 0.6622107391597394
